In [25]:
import cv2
import numpy as np
import imutils
import os

### Resize the Image

In [26]:
image = cv2.imread("./images/test_4.jpg")
height = image.shape[0]
width = image.shape[1]

ratio = 0.2  # Make it smaller for easy processing
width = int(ratio * width)
height = int(ratio * height)
orig = image.copy()
orig_resized = imutils.resize(image, height=height)
image = imutils.resize(image, height=height)
cv2.imshow("Image",image)
cv2.waitKey(0)


-1

### Turn it to Gray Scale

In [27]:

# # Strategy #1
# gray_scaled = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

# # Blur to smoothen out and remove unnecessary pixels
# gray_scaled = cv2.GaussianBlur(gray_scaled,(5,5),0)  


#Strategy #2
gray_scaled = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

cv2.imshow("Gray Scaled",gray_scaled)
cv2.waitKey(0)

gray_scaled = cv2.GaussianBlur(gray_scaled, (7,7), 0)

cv2.imshow("Blurred Gray Scaled",gray_scaled)
cv2.waitKey(0)

# dilate helps to remove potential holes between edge segments
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))

dilated = cv2.morphologyEx(gray_scaled, cv2.MORPH_CLOSE, kernel)
cv2.imshow("Dilated",dilated)
cv2.waitKey(0)

# Find the edges if use, strategy #1 change dilated to gray_scaled
edged_img = cv2.Canny(dilated,0,84)

cv2.imshow("Edges",edged_img)
cv2.waitKey(0)


-1

### Find Countours

In [28]:
#RETR LIST IS BETTER
contours, hierarchy = cv2.findContours(edged_img.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
if len(contours) == 3:
    contours = contours[0] 
# Now grab the contours using imutils
contours = imutils.grab_contours((contours, hierarchy))

# Sort the contours based on size and pick the top 5
contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5]

# loop over the contours
paper_outline = None
for contour in contours:
    perimeter = cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, 0.01 * perimeter, True)

    # Filter contours by area
    area = cv2.contourArea(contour)
    if area < 5000:  # Minimum area threshold
        continue

    # Check aspect ratio
    x, y, w, h = cv2.boundingRect(approx)
    aspect_ratio = float(w) / h
    if 0.5 < aspect_ratio < 2.0 and len(approx) == 4:
        paper_outline = approx
        break



### Draw Contours

In [29]:
cv2.drawContours(image,[paper_outline],-1,(255,255,0),2)  
cv2.imshow("Found outline", image)  
cv2.waitKey(0)
    

-1

### Get Corner Points

In [30]:
def order_points(points):
   # initialize a list of co-ordinates that will be ordered
   rectangle = np.zeros((4, 2), dtype="float32")
   sum_points = points.sum(axis=1)
   rectangle[0] = points[np.argmin(sum_points)]  # top-left
   rectangle[2] = points[np.argmax(sum_points)]  # bottom-right
   diff_points = np.diff(points, axis=1)
   rectangle[1] = points[np.argmin(diff_points)]  # top-right
   rectangle[3] = points[np.argmax(diff_points)]  # bottom-left
   return rectangle

### Warp the Image

In [31]:
def set_four_points(image, pts):
    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    # compute the width and height of the new image
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    dst = np.array([[0, 0], [maxWidth - 1, 0], [maxWidth - 1, maxHeight - 1], [0, maxHeight - 1]], dtype="float32")

    transform_matrix = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, transform_matrix, (maxWidth, maxHeight))

    return warped

In [32]:
warped = set_four_points(orig, paper_outline.reshape(4, 2) * (1 / ratio))

print("Image Reset in progress")
cv2.imshow("Original", imutils.resize(orig, height))
cv2.imshow("Scanned", imutils.resize(warped, height))
cv2.waitKey(0)

Image Reset in progress


-1

### Enhance the Image

In [ ]:

def enhance_image(warped):
    contrast_factor = 1.1  # Increase this value to increase contrast further
    enhanced_contrast = cv2.convertScaleAbs(warped, alpha=contrast_factor, beta=0)
    
    brightness_factor = -5  # You can adjust this value to lighten or darken the image
    adjusted_brightness = cv2.convertScaleAbs(enhanced_contrast, alpha=1, beta=brightness_factor)
    
    saturation_factor = 1.1
    # 1. Convert the image from BGR to HSV
    hsv = cv2.cvtColor(adjusted_brightness, cv2.COLOR_BGR2HSV)
    
    # 2. Increase saturation by multiplying the saturation channel (index 1) by a factor
    h, s, v = cv2.split(hsv)
    
    # Increase the saturation
    s = cv2.multiply(s, saturation_factor)
    s = np.clip(s, 0, 255)  # Ensure the values stay within the valid range [0, 255]
    
    # 3. Merge the channels back
    hsv_enhanced = cv2.merge([h, s, v])
    
    # 4. Convert back to BGR color space
    enhanced_image = cv2.cvtColor(hsv_enhanced, cv2.COLOR_HSV2BGR)

    return enhanced_image
# Enhance the warped image
enhanced_warped = enhance_image(warped)

# Show the original and enhanced images
cv2.imshow("Original Warped Image", imutils.resize(warped, height))
cv2.imshow("Enhanced Warped Image", imutils.resize(enhanced_warped, height))
cv2.waitKey(0)

-1

### Extract Text

In [ ]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'
def extract_text(warped_image):
    gray = cv2.cvtColor(warped_image, cv2.COLOR_BGR2GRAY)
    
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    kernel = np.ones((1, 1), np.uint8)
    thresh = cv2.dilate(thresh, kernel, iterations=1)  # Optional step, depending on your image

    text = pytesseract.image_to_string(thresh)

    return text

extracted_text = extract_text(enhanced_warped)

# Print the extracted text
print("Extracted Text:")
print(extracted_text)

Extracted Text:
Whats Lorem Ipsum?

Forem Epsum is simply dummy text of the Poot and t pesetting madustry Lorem Epsum has
been the industey's standard dummy text ever since the PS00s, when an unknown printer took a
talles of type and scrambled it to make at pe specamen book Tt has survived not only five
centuries, bit also the leap into electronic ts pesetting. rematning essentially unchanged. ft was
popularised m the LOeOs with the release of Letraset sheets containing Lorem [psum passages,
and mere recenth wath desktop publishing software like Aldus PageMaker including versions of

Lorem Ipsum
Whey do we use it?

Its a long established fact that a reader will be distracted by the readable content of a page when
looking at its layout The point of using Lorem Ipsum is that it has a more-or-less normal
distribution of letters, as opposed to using "Content here, content here’. making it look like
readable English. Many desktop publishing packages and web page editors now use lorem
Ipsum 

In [35]:
upload_folder = "./uploads"
if not os.path.exists(upload_folder):
    os.makedirs(upload_folder)

text_output_path = os.path.join(upload_folder,"extracted_text.txt")

# Save the warped (scanned) image
img_output_path = os.path.join(upload_folder, "scanned_image.jpg")
cv2.imwrite(img_output_path, enhanced_warped)
with open(text_output_path, "w") as file:
    file.write(extracted_text)
